In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

pd.options.display.max_columns = 100
pd.options.display.max_rows = 150

In [2]:
df_train = pd.read_csv("data/competition_data/train.csv")
df_train = df_train.sample(frac=1).reset_index(drop=True)

df_test = pd.read_csv("data/competition_data/test.csv")

df_train

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,249739,0,0,1,T,Y,Red,Trapezoid,Hamster,Russia,Oboe,6c90f81cd,71c68195d,d79956ebd,2d10dbc79,39bb4f59a,3,Grandmaster,Boiling Hot,d,L,Ed,7,1,0
1,29864,0,0,0,T,Y,Green,Star,Axolotl,Russia,Bassoon,63f642dd9,7d8bc814a,564a11aa1,f8407e5b6,8c8a56f3d,1,Novice,Cold,h,B,Ml,3,2,0
2,249597,0,0,1,F,Y,Green,Polygon,Snake,Costa Rica,Piano,0de5598a9,aed128d19,a6f3a3931,1a2fd4686,a1643d4d4,1,Expert,Freezing,a,U,Er,1,1,0
3,31709,1,0,1,F,N,Red,Triangle,Snake,Russia,Piano,f7821e391,d4fd01769,a2507f97d,677a0c809,3f2a6b542,3,Contributor,Cold,a,C,qw,3,4,0
4,173755,0,0,0,T,Y,Red,Circle,Lion,China,Bassoon,e9f44a509,5a49dd8f6,d82d4d997,9c584d3a4,98dd97c71,1,Grandmaster,Lava Hot,j,L,KZ,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,101582,0,0,0,T,N,Blue,Circle,Snake,Russia,Piano,fd04a970f,612641073,2488ac2fa,f4aa5e97b,9b1570ce2,1,Novice,Boiling Hot,k,S,cW,4,7,1
299996,269319,0,0,1,T,N,Red,Trapezoid,Lion,Canada,Theremin,9bb7ea2da,a20513779,479b4bade,e5fcb5ae0,bd1c2871a,1,Grandmaster,Boiling Hot,n,T,Bb,7,9,1
299997,8079,0,0,0,T,Y,Blue,Trapezoid,Hamster,Finland,Oboe,e70a6270d,36df19a6e,4a7761d5f,91c3125ab,48039a325,1,Grandmaster,Freezing,f,S,kE,2,11,1
299998,59767,0,1,0,T,N,Blue,Star,Lion,Costa Rica,Bassoon,05950689f,1869e63b0,07bba85ec,9b05fba81,5a2e32b63,1,Contributor,Lava Hot,a,W,gJ,4,10,0


In [3]:
y_train = df_train["target"]

y_train

0         0
1         0
2         0
3         0
4         0
         ..
299995    1
299996    1
299997    1
299998    0
299999    1
Name: target, Length: 300000, dtype: int64

# Logistic Regression on category codes

Expected to perform bad. Used as a benchmark for later one hot encoding

In [4]:
from sklearn.preprocessing import MinMaxScaler
    
def process_features_cats_to_codes(df):
    df_out = df.copy()
    
    # binary features to 0, 1
    df_out["bin_3"] = df_out["bin_3"].map({"T": 1, "F": 0})
    df_out["bin_4"] = df_out["bin_4"].map({"Y": 1, "N": 0})

    # ordinal features to int
    for field in ["ord_1", "ord_2", "ord_3", "ord_4", "ord_5"]:
        df[field] = pd.Categorical(df[field])
        df_out[field] = df[field].cat.codes
        
    # nominal features to int (same as for ordinal, but let's separate to keep us some options)
    for field in ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']:
        df[field] = pd.Categorical(df[field])
        df_out[field] = df[field].cat.codes
        
    # scale the feature
    scaler = MinMaxScaler()
    df_out = pd.DataFrame(scaler.fit_transform(df_out), columns=df_out.columns)
    
    return df_out


df_train_codes = process_features_cats_to_codes(df_train)

del df_train_codes["target"]
del df_train_codes["id"]

df_train_codes

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0.0,0.0,1.0,1.0,1.0,1.0,0.8,0.6,1.0,0.333333,0.452489,0.437620,0.838392,0.176152,0.224124,1.0,0.50,0.0,0.214286,0.44,0.089005,1.000000,0.000000
1,0.0,0.0,0.0,1.0,1.0,0.5,0.6,0.0,1.0,0.000000,0.434389,0.479846,0.317473,0.970190,0.544908,0.0,1.00,0.2,0.500000,0.04,0.225131,0.333333,0.090909
2,0.0,0.0,1.0,0.0,1.0,0.5,0.2,1.0,0.4,0.666667,0.063348,0.662188,0.630845,0.112014,0.623539,0.0,0.25,0.4,0.000000,0.80,0.099476,0.000000,0.000000
3,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.666667,0.972851,0.821497,0.609516,0.407407,0.245659,1.0,0.00,0.2,0.000000,0.08,0.848168,0.333333,0.272727
4,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.8,0.2,0.000000,0.914027,0.351248,0.844135,0.614724,0.590902,0.0,0.50,0.8,0.642857,0.44,0.183246,0.166667,0.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.666667,1.000000,0.383877,0.127153,0.954833,0.599249,0.0,1.00,0.0,0.714286,0.72,0.513089,0.500000,0.545455
299996,0.0,0.0,1.0,1.0,0.0,1.0,0.8,0.8,0.0,1.000000,0.651584,0.629559,0.253486,0.897019,0.733139,0.0,0.50,0.0,0.928571,0.76,0.026178,1.000000,0.727273
299997,0.0,0.0,0.0,1.0,1.0,0.0,0.8,0.6,0.6,0.333333,0.891403,0.220729,0.264971,0.569557,0.279716,0.0,0.50,0.4,0.357143,0.72,0.680628,0.166667,0.909091
299998,0.0,1.0,0.0,1.0,0.0,0.0,0.6,0.8,0.4,0.000000,0.009050,0.090211,0.036095,0.606143,0.351419,0.0,0.00,0.8,0.000000,0.88,0.612565,0.500000,0.818182


In [5]:
from sklearn.linear_model import LogisticRegression

In [6]:
# for training benchmark simply split training set in half

i = df_train_codes.shape[0]
df_train_codes1 = df_train_codes.copy()[:i // 2]
df_train_codes2 = df_train_codes.copy()[i // 2:]
y_train1=y_train[:i // 2]
y_train2=y_train[i // 2:]

In [7]:
model = LogisticRegression()

def fit_and_score(model, df_train, y_train, df_test, y_test):
    model = model.fit(df_train, y_train)
    
    y_score_train = [s[1] for s in model.predict_proba(df_train)]
    y_score_test = [s[1] for s in model.predict_proba(df_test)]

    print(f"""
        Score on training set: {roc_auc_score(y_true=y_train, y_score=y_score_train)}
        Score on test set: {roc_auc_score(y_true=y_test, y_score=y_score_test)}
    """)
    
    return model

model = fit_and_score(model, df_train_codes1, y_train1, df_train_codes2, y_train2)



        Score on training set: 0.717428296129853
        Score on test set: 0.7161564198869559
    


In [8]:
"""Ok, good start, let's do one-hot encoding"""

"Ok, good start, let's do one-hot encoding"

# Logistic Regression on One Hot encoded features

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [24]:
x = 300000

df_train_oh = df_train.copy()[:x]

del df_train_oh["id"]
del df_train_oh["target"]

oh = OneHotEncoder()

df_train_oh = oh.fit_transform(df_train_oh)
y_train_oh = y_train[:x]

df_train_oh.shape

(300000, 16461)

In [25]:
model = LogisticRegression(
    max_iter = 400,
    penalty='elasticnet',
    l1_ratio=1,
    C=.1,
    solver='saga',
    verbose=3
)

model = fit_and_score(
    model,
    df_train_oh[:x // 2],
    y_train_oh[:x // 2],
    df_train_oh[x // 2:],
    y_train_oh[x // 2:]
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 25 epochs took 19 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.5s finished



        Score on training set: 0.7965108136580874
        Score on test set: 0.786715973330103
    


In [26]:
"""Looks nice, let's encode the ordinal features linearly"""

"Looks nice, let's encode the ordinal features linearly"

# one hot the ordinals, linear the nominals

In [27]:
df_train.columns

Index(['id', 'bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1',
       'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9',
       'ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month',
       'target'],
      dtype='object')

In [28]:
df_train_ord = df_train.copy()[['ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']]
df_train_oh_nom = df_train.copy()[
    ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1',
       'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'day', 'month']
]

oh = OneHotEncoder()

df_train_oh_nom = oh.fit_transform(df_train_oh_nom)
y_train_oh = y_train[:x]

df_train_oh_nom.shape

(300000, 16214)

In [38]:
"""TODO: encode the ordinaly to ascending ints and scale to [0, 1]"""

df_train_ord

,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5
0,3,Grandmaster,Boiling Hot,d,L,Ed
1,1,Novice,Cold,h,B,Ml
2,1,Expert,Freezing,a,U,Er
3,3,Contributor,Cold,a,C,qw
4,1,Grandmaster,Lava Hot,j,L,KZ
...,...,...,...,...,...,...
299995,1,Novice,Boiling Hot,k,S,cW
299996,1,Grandmaster,Boiling Hot,n,T,Bb
299997,1,Grandmaster,Freezing,f,S,kE
299998,1,Contributor,Lava Hot,a,W,gJ


In [29]:
model = LogisticRegression(
    max_iter = 400,
    penalty='elasticnet',
    l1_ratio=1,
    C=.1,
    solver='saga',
    verbose=3
)

model = fit_and_score(
    model,
    df_train_oh_nom[:x // 2],
    y_train_oh[:x // 2],
    df_train_oh_nom[x // 2:],
    y_train_oh[x // 2:]
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 27 epochs took 18 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.6s finished



        Score on training set: 0.6913701509211135
        Score on test set: 0.6810894768208013
    
